# Item-Item Top-N Recommendations

In this excercise we will implement a simple top-N recommender, evaluate the algorithms, and then call algorithms from the Surprise package. In top-N recommendations the algorithm is requested to produce a list of N items that the user will be interested in. 
In this particular execercise we will work with an escape room dataset.

First, let's load the dataset, which is already split by time into a training set and a test set:

In [1]:
import numpy as np
import pandas as pd

train_set_path = 'resources//train_numerized_with_anon.csv'
test_set_path = 'resources//test_numerized_with_anon.csv'

train_set = pd.read_csv(train_set_path, parse_dates=[3], index_col='index')
test_set = pd.read_csv(test_set_path, parse_dates=[3], index_col='index')

users_in_train = train_set.userID.unique()
test_set = test_set[test_set.userID.isin(users_in_train)]

We can take a look at the structure of the dataset:

In [2]:
train_set.head()

,userID,itemID,rating,timestamp
index,,,,
0,0,0,10,11/11/2015
1,1,0,10,11/11/2015
2,2,0,8,11/11/2015
3,3,0,10,11/11/2015
4,4,0,10,11/11/2015


## Part 1: Recommend Most Popular Items (_max score - 20 points_)

Now we can begin implementing the our first algorithm, that recommends to the user the list of most popular items. Although this is not a personalized approach, in many cases, this is not a bad idea - popular items are popular because everybody choose them, so there is a high likelihood that recommended popular items will be indeed chosen by the user.

In the code below, fill in the missing parts. The algorithm has a training method, where item popularity is computed, and a recommendation method, where the list of popular items.

In [3]:
class MostPopular:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None

    def learn_model(self, train_set):
        self.train_set = train_set
        #1) Add code to set the item_ratings_sorted to the list of items in the training set, 
        #ordered by decreasing popularity (i.e., the number of users who have chosen an item)
        counts = self.train_set.groupby('itemID').count()['userID']
        self.counts = counts.sort_values(ascending= False)

    def get_top_n_recommendations(self, test_set, top_n):
        result = {}
        already_ranked_items_by_user = self.train_set.groupby('userID')['itemID'].apply(list)
        
        
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            
            top_movies = self.counts.sort_values(ascending= False).index
            user_movies = already_ranked_items_by_user[userID]
            users_top_movies_idx = list(set(top_movies) - set(user_movies))
            users_top_movies = self.counts[users_top_movies_idx].sort_values(ascending = False).index #Use the filtered movie list, reorder by the popularity, and return the indexes.


            result[str(userID)] = list(users_top_movies[:top_n]) #return the top_n movies for each user
            
            #2) Add to the result the top N items from the popular list
            #3) Remove items that the user has already chosen in the training set (already_ranked_items_by_user)
        return result

    def clone(self):
        pass


Now we can call the most popular algorithm to deliver a list of reocmmendations. The code below prints the list of top 5 recommended items for user with ID 431.

In [4]:
popular = MostPopular()
popular.learn_model(train_set)
popular_recs = popular.get_top_n_recommendations(test_set,top_n=5)
print(popular_recs['431'])
assert popular_recs['431'] == [53, 26, 68, 85, 16], 'Wrong computation of popular items'

[53, 26, 68, 85, 16]


## Part 2 - Item-Item Recommendations (_max score - 30 points_)

We now learn a slightly more sophisticated model, that uses item-item similarities. Given such a similarity score, we can recommend to a user items that are most similar to the items that the user has chosen in the past. One such useful similarity metric is the Jaccard coefficient. For two items i1 and i2, the Jaccard similarity is the number of users who have chosen both i1 and i2, divided by the number of users who have chosen either i1 or i2. That is, given the list of users who have chosen i1 and the list of users who have chosen i2, the Jaccard similarity is the intersection of the lists, divided by the union of the lists.

In practice, to expedite the recommendation process, and hence reduce online latency, we will compute the item-item co-occurence matrix in the model learning phase. Then, online, when recommendations are requested, we only need to compute for each item that the user has already chosen in the past, the Jaccard scores for the other items.

As the user has chosen several items in the past, we need to aggregate the Jaccard scores. That is, if the user has previously chosen i1 and i2, item i3 has two scores J(i1,i3) and J(i2,i3), and an aggregation of the scores is needed. There are two popular aggregation functions - sum and max. Empirically, max typically has better perfromance.

Fill in the missing parts in the code below.

In [8]:
from tqdm import tqdm
import numpy as np
import operator
import itertools
class Jaccard:

    def __init__(self):
        self.item_ratings_sorted = None
        self.train_set = None
        self.item_item_counts = dict()
        self.item_counts = None

    def learn_model(self, train_set):
        print('Started training')
        self.train_set = train_set
        self.item_counts =  self.train_set.groupby('itemID')['userID'].agg('count') #count of how many users watched each movie
        
        maxpair = 0
        pbar = tqdm(total=len(train_set.userID.unique())) #progress bar
        
        #Iterating over all item pairs is inefficient. It is better to iterate only over pairs of items that were chosen together.
        #Instead, we will iterate over the users, and for each user, and each two items that the user has chosen, increase the count
        for u in train_set.userID.unique():
            pbar.update(1)
            userData = self.train_set[self.train_set.userID == u] #dataframe of all rankings per user
            
            #4) For each pair of items in the user data - increase the counts in self.item_item_counts
            ordered_values =sorted(userData.itemID.values)
            user_pairs = list(itertools.combinations(ordered_values, 2))
            # user_pairs =set()
 
            # # Make all possible pairs
            # for i in userData.itemID.values:
            #     for j in userData.itemID.values:
            #         user_pairs.add(np.sort((i, j)))
                    
            for pair in user_pairs:
                self.item_item_counts.setdefault(pair,0)
                self.item_item_counts[pair] +=1
            

    def get_top_n_recommendations(self, test_set, top_n):
        print('Started computing recommendations')
        result = {}
        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)
        
        pbar = tqdm(total=len(test_set.userID.unique()))
        
        #For each user in the test set compute recommendations
        for userID in test_set.userID.unique():
            pbar.update(1)
            result[str(userID)] = []
            #maxvalues will maintain for each potential item to recommend its highest Jaccard score.
            maxvalues = dict()
            
            #4) Iterate over the items that the user has already chosen.
            #For each such item compute its Jaccard correlation to other items based on the item_item_counts.
            #Aggregate into maxvalues using max.
            #Avoid recommending items that the user has already chosen in the training set.
            for users_movie in already_ranked_items_by_users[userID]:
                for pair in self.item_item_counts.keys():
                    # if i in pair:
                    #     movie_a = self.item_counts[pair[0]]
                    #     movie_b = self.item_counts[pair[1]]

                    #     intersection_count = self.item_item_counts[pair]

                    #     score = intersection_count / (movie_a + movie_b)
                    #     maxvalues[pair[0]] = score
                    if users_movie in pair:
                        
                        if users_movie == pair[0]: #Movie b is the one we are getting the scores of, based on their relation to a.
                                                 # Some values can either be the first element in the dict key, or the second element. We want to make sure
                            movie_a = pair[0]    # to account for all keys that that the user's_movie are in.
                            movie_b = pair[1]    #Meaning If we are finding 28: (28, 0) means that 0 is coorelated to 28. But we also 
                        else:                    # want to find where it is (1, 21), to account for 1 being coorelated to 21 as well.
                            movie_a = pair[1]
                            movie_b = pair[0]


                        movie_a_count = self.item_counts[movie_a] #Total amount of times User_movie is are rated.
                        movie_b_count = self.item_counts[movie_b] #Total amount of time the Jaccard movie is rated

                        intersection_count = self.item_item_counts[pair]

                        score = intersection_count / (movie_a_count + movie_b_count) #calculate jaccard
                        # maxvalues.setdefault(movie_b, 0) #
                        maxvalues[movie_b] = score #For each movie coorelated to User_Movie, return the jaccard score
                
            for watched in already_ranked_items_by_users[userID]: #Remove keys which have already been viewed by the user.
                for key in maxvalues.keys():
                    if watched == key:
                        maxvalues[key] = 0 #We can just set the value to zero, since we are sorting the values later to filter anyways.
            maxvalues = {key:value for key, value in maxvalues.items() if value !=0} #Remove all keys where the value is = to 0.

            #Now we just take the top N items by decreasing Jaccard
            top_list = sorted(maxvalues.items(), key=lambda kv : -kv[1])
            i=0
            j=0
            while i < top_n and j < len(top_list):
                itemID = top_list[j][0]
                # print(itemID)
                j = j + 1
                result[str(userID)].append(itemID)
                i = i + 1
            
        pbar.close()
        print('Done computing recommendations')
        return result

    
    def clone(self):
        pass


In [9]:
dic = {2:5, 3:0}

b = {key:value for key, value in dic.items() if value !=0}
b

{2: 5}

In [10]:
jaccard = Jaccard()
jaccard.learn_model(train_set)
jaccard_recs = jaccard.get_top_n_recommendations(test_set,top_n=5)

Started training


100%|██████████| 20197/20197 [00:16<00:00, 1257.47it/s]


Started computing recommendations


100%|██████████| 227/227 [04:10<00:00,  1.10s/it]

Done computing recommendations


The code below trains a Jaccard model and generates recommendations. Training will take a while, as we need to iterate over all users, and for each user go over her items in quadratic time.

As a side note - as computing the ite-item counts takes a while (especially with Python), we are using here the progress bar from the tqdm package (https://pypi.org/project/tqdm/). You need to install tqdm, or remove the progress bar, which would of course is not needed for the algorithm to run.

## Part 3 - Comparing the Algorithms (_max score - 30 points_)

We now want to compare the recommendation lists to see which one is better. In top-N recommendations it is popular to computer the Precision@N metric - the portion of recommended items that were chosen by users in the test set. This is typically a reasonable metric for real systems, where one wants to optimize the number of recommended items that are chosen.

We compute Precision@N by comparing the number of recommendations chosen by the users, divided by the number of overall recommendations.

Fill in the missing parts in the code below:

In [11]:
def compute_precision(test_set, recommendations):
    #hits is the number of items that were recommended and chosen
    hits = 0
    #recs is the total number of recommended items
    recs = 0
    
    for u in test_set.userID.unique():
        userData = test_set[test_set.userID == u]
        userRecs = recommendations.get(str(u))
        #5) Compute here the number of hits. Update hits and recs accordingly.
        hits+= len(set(userData.itemID) & set(userRecs))
        recs+= len(userRecs)
    return hits / recs
        
    

In [12]:
p1 = compute_precision(test_set, jaccard_recs)
p2 = compute_precision(test_set, popular_recs)
print("Jaccard=", p1, "  Popularity=", p2)

Jaccard= 0.04669603524229075   Popularity= 0.027312775330396475


The precision values for this dataset may seem pretty low, but this is typical for many top-N problems. It is important not to compute metrics that hide the true values, such as AUC, but to acknowledge the perfromance of the system in the application.

## Part 4 - Calling Algorithms from the Surprise Package (_max score - 20 points_)

There are many existing recommendation algorithms available. We will now see how we can call algorithms from the Surprise package. 

The code below adds a wrapper around the algorithm to transform the resulting recommendations into our desired format.

#### NOTE: 
To run the code below you first have to install _surprise_ (http://surpriselib.com/). _Surprise_ requires scipy >=1.0, so update if needed.

To install: pip install scikit-surprise or, if you're using anaconda:  conda install -c conda-forge scikit-surprise).

If you have a problem in installing the _surprise_ library locally, you can use Google Colab (but there can be an OOM problem with KNNBasic algorithm - in this case, just skip this algorithm).

In [13]:
import sys, string, os
import pandas as pd
import itertools
from tqdm import tqdm
import numpy as np
import operator
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import PredefinedKFold
from surprise.prediction_algorithms import *



class SurpriseRecMethod():

    #method will be the specific Surprise algorithm that we will call
    def __init__(self, method):
        self.method = method

    def fit(self, train_set):
        self.train_set = train_set


    def get_top_n_recommendations(self, test_set, top_n):
        self.test_set = test_set

        #Surprise requires a slightly different input data format, so we use two different CSVs
        test_path_tmp = "resources//test_file.csv"
        train_path_tmp = "resources//train_file.csv"

        self.train_set.to_csv(train_path_tmp, index=False, header=False)
        self.test_set.to_csv(test_path_tmp, index=False, header=False)

        fold_files = [(train_path_tmp, test_path_tmp)]
        reader = Reader(rating_scale=(1, 10), line_format='user item rating', sep=',')
        data = Dataset.load_from_folds(fold_files, reader=reader)

        for trainset, testset in PredefinedKFold().split(data):
            self.method.fit(trainset)

        already_ranked_items_by_users = self.train_set.groupby('userID')['itemID'].apply(list)

        recommendations = {}
        pbar = tqdm(total=len(self.test_set.userID.unique()))
        for userID in self.test_set.userID.unique():
            pbar.update(1)

            if userID not in self.train_set.userID.unique():
                recommendations[str(userID)] = []
                continue

            items_expected_ranking = {}
            for itemID in self.train_set.itemID.unique():
                if itemID in already_ranked_items_by_users[userID]:
                    continue
                #We call here the specific Surprise method that we use for this model
                #The method predicts a score for a given item
                predicted = self.method.predict(str(userID), str(itemID), clip=False)
                items_expected_ranking[itemID] = predicted[3]
                
            #Now we just sort by decreasing scores and take the top N
            sorted_predictions = sorted(items_expected_ranking.items(), key=operator.itemgetter(1))
            sorted_predictions.reverse()
            sorted_predictions = [x[0] for x in sorted_predictions]
            user_recommendations = sorted_predictions[:top_n]
            recommendations[str(userID)] = user_recommendations
        pbar.close()
        return recommendations


The code below calls the package with the SlopeOne algorithm.

In [14]:
modelSlopeOne = SurpriseRecMethod(SlopeOne())
modelSlopeOne.fit(train_set)
recSlopeOne = modelSlopeOne.get_top_n_recommendations(test_set, 5)
p3 = compute_precision(test_set,recSlopeOne)

C:\Users\sfrie\anaconda3\envs\py4dp\lib\site-packages\ipykernel_launcher.py:39: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
100%|██████████| 227/227 [00:03<00:00, 61.64it/s]


The code below calls the package with a nearest neighbor user-item recommendation method.

In [15]:
modelKNNUser = SurpriseRecMethod(KNNBasic(sim_options={'name': 'cosine', 'user_based': True}))
modelKNNUser.fit(train_set)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_set, 5)
p4 = compute_precision(test_set,recKNNUser)

Computing the cosine similarity matrix...
Done computing similarity matrix.


100%|██████████| 227/227 [00:12<00:00, 18.68it/s]


Let us look at the results of all algorithms together:

In [16]:
pd.DataFrame.from_dict({'Jaccard':p1,'Popularity':p2,'SlopeOne':p3,'User KNN':p4}, orient='index',columns=['Precision'])

,Precision
Jaccard,0.046696
Popularity,0.027313
SlopeOne,0.032599
User KNN,0.075771


Try the NMF (non-negative matrix factorization) algorithms from the package - https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.NMF.


In [17]:
#Your code here
modelKNNUser = SurpriseRecMethod(NMF())
modelKNNUser.fit(train_set)
recKNNUser = modelKNNUser.get_top_n_recommendations(test_set, 5)
p5 = compute_precision(test_set,recKNNUser)
p5

100%|██████████| 227/227 [00:01<00:00, 115.20it/s]


0.03259911894273128

For this particular dataset, the user nearest neighbor approach worked best. Hence, should we need to choose a method to put online, we should go with this method.